In [1]:
import pandas as pd
df = pd.read_csv("AnimeList.csv")
pd.set_option('display.max_columns', None)

## Data Cleaning

In [2]:
columns_to_remove = ['anime_id',
                     'title', 
                     'title_english', 
                     'title_japanese', 
                     'title_synonyms',
                     'image_url', 'status', 
                     'airing', 
                     'aired_string', 
                     'aired', 
                     'scored_by', 
                     'rank', 
                     'popularity', 
                     'favorites', 
                     'background', 
                     'broadcast', 
                     'related', 
                     'opening_theme', 
                     'ending_theme',
                     'premiered', 
                     'studio', 
                     'producer']
df = df.drop(columns_to_remove, axis=1)

Above were the columns we didn't need.

In [3]:
df.dropna(subset=['rating'], inplace=True)
df.drop('licensor', axis=1, inplace=True)
df.dropna(subset=['genre'], inplace=True)

## Data Preprocessing

In [4]:
df.dtypes

type         object
source       object
episodes      int64
duration     object
rating       object
score       float64
members       int64
genre        object
dtype: object

In [5]:
df.head()

,type,source,episodes,duration,rating,score,members,genre
0,TV,Manga,12,24 min. per ep.,PG-13 - Teens 13 or older,7.63,283882,"Comedy, Supernatural, Romance, Shounen"
1,TV,Manga,26,24 min. per ep.,PG-13 - Teens 13 or older,7.89,204003,"Comedy, Parody, Romance, School, Shounen"
2,TV,Manga,51,24 min. per ep.,PG - Children,7.55,70127,"Comedy, Magic, School, Shoujo"
3,TV,Original,38,16 min. per ep.,PG-13 - Teens 13 or older,8.21,93312,"Comedy, Drama, Magic, Romance, Fantasy"
4,TV,Manga,25,24 min. per ep.,PG-13 - Teens 13 or older,8.67,182765,"Comedy, Drama, Romance, Shounen"


In [6]:
pd.Series(df['type'].unique())

0         TV
1      Movie
2      Music
3        OVA
4        ONA
5    Special
6    Unknown
dtype: object

In [6]:
genres = df['genre'].str.get_dummies(sep=', ')
df = pd.concat([df, genres], axis=1)
df.drop('genre', axis=1, inplace=True)
columns_to_encode = ['type', 'source', 'duration', 'rating']
encoded_df = pd.get_dummies(df, columns=columns_to_encode)
df = pd.concat([df, encoded_df], axis=1)
new_columns_to_drop = ['type', 'source', 'duration', 'rating']
df = df.drop(new_columns_to_drop, axis=1)

In [7]:
df.head()

,episodes,score,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,episodes,score,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,type_Unknown,source_4-koma manga,source_Book,source_Card game,source_Digital manga,source_Game,source_Light novel,source_Manga,source_Music,source_Novel,source_Original,source_Other,source_Picture book,source_Radio,source_Unknown,source_Visual novel,source_Web manga,duration_1 hr.,duration_1 hr. 1 min.,duration_1 hr. 10 min.,duration_1 hr. 11 min.,duration_1 hr. 12 min.,duration_1 hr. 13 min.,duration_1 hr. 14 min.,duration_1 hr. 14 min. per ep.,duration_1 hr. 15 min.,duration_1 hr. 15 min. per ep.,duration_1 hr. 16 min.,duration_1 hr. 16 min. per ep.,duration_1 hr. 17 min.,duration_1 hr. 17 min. per ep.,duration_1 hr. 18 min.,duration_1 hr. 19 min.,duration_1 hr. 2 min.,duration_1 hr. 2 min. per ep.,duration_1 hr. 20 min.,duration_1 hr. 21 min.,duration_1 hr. 22 min.,duration_1 hr. 23 min.,duration_1 hr. 24 min.,duration_1 hr. 24 min. per ep.,duration_1 hr. 25 min.,duration_1 hr. 26 min.,duration_1 hr. 27 min.,duration_1 hr. 28 min.,duration_1 hr. 29 min.,duration_1 hr. 3 min.,duration_1 hr. 3 min. per ep.,duration_1 hr. 30 min.,duration_1 hr. 30 min. per ep.,duration_1 hr. 31 min.,duration_1 hr. 32 min.,duration_1 hr. 33 min.,duration_1 hr. 34 min.,duration_1 hr. 35 min.,duration_1 hr. 35 min. per ep.,duration_1 hr. 36 min.,duration_1 hr. 36 min. per ep.,duration_1 hr. 37 min.,duration_1 hr. 38 min.,duration_1 hr. 39 min.,duration_1 hr. 4 min.,duration_1 hr. 40 min.,duration_1 hr. 41 min.,duration_1 hr. 42 min.,duration_1 hr. 43 min.,duration_1 hr. 44 min.,duration_1 hr. 45 min.,duration_1 hr. 46 min.,duration_1 hr. 47 min.,duration_1 hr. 48 min.,duration_1 hr. 49 min.,duration_1 hr. 5 min.,duration_1 hr. 5 min. per ep.,duration_1 hr. 50 min.,duration_1 hr. 51 min.,duration_1 hr. 52 min.,duration_1 hr. 52 min. per ep.,duration_1 hr. 53 min.,duration_1 hr. 54 min.,duration_1 hr. 55 min.,duration_1 hr. 56 min.,duration_1 hr. 57 min.,duration_1 hr. 58 min.,duration_1 hr. 59 min.,duration_1 hr. 6 min.,duration_1 hr. 7 min.,duration_1 hr. 7 min. per ep.,duration_1 hr. 8 min.,duration_1 hr. 9 min.,duration_1 hr. per ep.,duration_1 min.,duration_1 min. per ep.,duration_10 min.,duration_10 min. per ep.,duration_10 sec.,duration_10 sec. per ep.,duration_11 min.,duration_11 min. per ep.,duration_12 min.,duration_12 min. per ep.,duration_12 sec.,duration_12 sec. per ep.,duration_13 min.,duration_13 min. per ep.,duration_13 sec.,duration_14 min.,duration_14 min. per ep.,duration_14 sec.,duration_14 sec. per ep.,duration_15 min.,duration_15 min. per ep.,duration_15 sec.,duration_15 sec. per ep.,duration_16 min.,duration_16 min. per ep.,duration_16 sec.,duration_16 sec. per ep.,duration_17 min.,duration_17 min. per ep.,duration_17 sec.,duration_18 min.,duration_18 min. per ep.,duration_18 sec.,duration_19 min.,duration_19 min. per ep.,duration_19 sec. per ep.,duration_2 hr.,duration_2 hr. 1 min.,duration_2 hr. 10 min.,duration_2 hr. 11 min.,duration_2 hr. 12 min.,duration_2 hr. 14 min.,duration_2 hr. 15 min.,duration_2 hr. 16 min.,duration_2 hr. 17 min.,duration_2 hr. 19 min.,duration_2 hr. 2 min.,duration_2 hr. 20 min.,duration_2 hr. 21 min.,duration_2 hr. 3 min.,duration_2 hr. 30 min.,duration_2 hr. 31 min.,duration_2 hr. 32 min.,duration_2 hr. 33 min.,duration_2 

In [8]:
from sklearn.model_selection import train_test_split

features = df.drop('score', axis=1)
target = df['score']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Check the shapes of the resulting sets
print("Training set shape - Features:", X_train.shape, "Target:", y_train.shape)
print("Testing set shape - Features:", X_test.shape, "Target:", y_test.shape)

Training set shape - Features: (11106, 417) Target: (11106, 2)
Testing set shape - Features: (2777, 417) Target: (2777, 2)


In [9]:
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

def train_and_evaluate_models(X_train, y_train, X_test, y_test):
    # Initialize regression models
    models = [
        LinearRegression(),
        Ridge(),
        Lasso(),
        RandomForestRegressor()
    ]
    
    # Initialize dictionaries to store evaluation metrics
    mse_scores = {}
    r2_scores = {}
    explained_var_scores = {}
    
    # Train and evaluate each model
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mse = mean_squared_error(y_test, y_pred)
        mse_scores[model.__class__.__name__] = mse
        
        r2 = r2_score(y_test, y_pred)
        r2_scores[model.__class__.__name__] = r2
        
        explained_var = explained_variance_score(y_test, y_pred)
        explained_var_scores[model.__class__.__name__] = explained_var
    
    return mse_scores, r2_scores, explained_var_scores

# Usage
mse_scores, r2_scores, explained_var_scores = train_and_evaluate_models(X_train, y_train, X_test, y_test)

In [10]:
# Print Mean Squared Error (MSE) scores for each model
print("Mean Squared Error (MSE) scores:")
for model_name, mse_score in mse_scores.items():
    print(f"{model_name}: {mse_score}")

# Print R-squared scores for each model
print("\nR-squared (Coefficient of Determination) scores:")
for model_name, r2_score in r2_scores.items():
    print(f"{model_name}: {r2_score}")

# Print Explained Variance scores for each model
print("\nExplained Variance scores:")
for model_name, explained_var_score in explained_var_scores.items():
    print(f"{model_name}: {explained_var_score}")

Mean Squared Error (MSE) scores:
LinearRegression: 0.8994392629480931
Ridge: 0.8968213235139456
Lasso: 1.5662267138444146
RandomForestRegressor: 0.6425363081961578

R-squared (Coefficient of Determination) scores:
LinearRegression: 0.47644992842850264
Ridge: 0.47797379161146536
Lasso: 0.08832297864924665
RandomForestRegressor: 0.6259892757619167

Explained Variance scores:
LinearRegression: 0.476494624081097
Ridge: 0.47802630418934666
Lasso: 0.08866073845698408
RandomForestRegressor: 0.6260035609592067


In [11]:
from sklearn.ensemble import RandomForestRegressor
from joblib import dump

# Assuming 'X_train' and 'y_train' are your training data
# Train RandomForestRegressor
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Save the trained model to a .pkl file
dump(rf_model, 'random_forest_regressor.pkl')

['random_forest_regressor.pkl']

In [12]:
df.columns

Index(['episodes', 'score', 'members', 'Action', 'Adventure', 'Cars', 'Comedy',
       'Dementia', 'Demons', 'Drama',
       ...
       'duration_8 min. per ep.', 'duration_9 min.', 'duration_9 min. per ep.',
       'duration_Unknown', 'rating_G - All Ages', 'rating_PG - Children',
       'rating_PG-13 - Teens 13 or older',
       'rating_R - 17+ (violence & profanity)', 'rating_R+ - Mild Nudity',
       'rating_Rx - Hentai'],
      dtype='object', length=419)

In [13]:
columns_list = df.columns.tolist()

In [14]:
print(columns_list)

['episodes', 'score', 'members', 'Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri', 'episodes', 'score', 'members', 'Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri